# Import Libraries

In [1]:
import json
from tqdm import tqdm
import pandas

import networkx as nx
from pyvis.network import Network

from operator import itemgetter


# Read log and display sample

In [2]:
filepath = r'C:\Users\A101889\Documents\GitHub\SecurityAnalytics\Playground\sample_dataset\mimikatz_logonpasswords_2020-08-07103224.json'

with open(filepath) as f:
    readfile = f.readlines()
    
sampledata = json.loads(readfile[0]) 
sampledata

{'tags': ['mordorDataset'],
 'TargetProcessGUID': '{9f85ce58-5a6a-5f2b-a900-000000000400}',
 '@version': '1',
 'EventType': 'INFO',
 'Version': 3,
 'ThreadID': 4208,
 'EventTime': '2020-08-07 10:32:22',
 'Task': 10,
 'AccountType': 'User',
 'SourceProcessGUID': '{9f85ce58-59dc-5f2b-1000-000000000400}',
 'CallTrace': 'C:\\windows\\SYSTEM32\\ntdll.dll+9fc24|C:\\windows\\System32\\KERNELBASE.dll+6c175|c:\\windows\\system32\\lsm.dll+e575|C:\\windows\\System32\\RPCRT4.dll+77803|C:\\windows\\System32\\RPCRT4.dll+db4a6|C:\\windows\\System32\\RPCRT4.dll+10d20|C:\\windows\\System32\\RPCRT4.dll+54a18|C:\\windows\\System32\\RPCRT4.dll+30490|C:\\windows\\System32\\RPCRT4.dll+2fe3b|C:\\windows\\System32\\RPCRT4.dll+221ff|C:\\windows\\System32\\RPCRT4.dll+2165a|C:\\windows\\System32\\RPCRT4.dll+20c21|C:\\windows\\System32\\RPCRT4.dll+20692|C:\\windows\\System32\\RPCRT4.dll+17465|C:\\windows\\SYSTEM32\\ntdll.dll+4f4d0|C:\\windows\\SYSTEM32\\ntdll.dll+50358|C:\\windows\\System32\\KERNEL32.DLL+17974|C:

In [3]:
# To get all list availalbe
uniqset = set()
for data in readfile:
    jsonobj = json.loads(data) 
    for keys in jsonobj.keys():
        tempp = keys.lower()
        matches = ["image"]
        if any(x in tempp for x in matches):
            uniqset.add(keys)

uniqset = list(uniqset)
sorted(uniqset)

['Image', 'ImageLoaded', 'ParentImage', 'SourceImage', 'TargetImage']

# Some Stats

In [4]:
# Top tools used

from collections import Counter

tools = []
for data in readfile:
    jsonobj = json.loads(data)
    for keys in jsonobj.keys():
        tempp = keys.lower()
        matches = ["image"]
        if any(x in tempp for x in matches):
            tools.append(str(jsonobj[keys]).split('\\')[-1])
Counter(tools).most_common(10)

[('svchost.exe', 614),
 ('powershell.exe', 109),
 ('lsass.exe', 67),
 ('dns.exe', 32),
 ('whoami.exe', 31),
 ('Microsoft.ActiveDirectory.WebServices.exe', 9),
 ('wmidcom.dll', 9),
 ('dpapi.dll', 9),
 ('perfctrs.dll', 9),
 ('DFSRs.exe', 9)]

In [5]:
tools = []
for data in readfile:
    jsonobj = json.loads(data)
    for keys in jsonobj.keys():
        tempp = keys.lower()
        matches = ["eventid"]
        if any(x in tempp for x in matches):
            tools.append(str(jsonobj[keys]).split('\\')[-1])
Counter(tools).most_common(10)

[('800', 2940),
 ('4103', 1695),
 ('10', 272),
 ('5156', 211),
 ('5158', 195),
 ('12', 154),
 ('3', 90),
 ('4658', 86),
 ('7', 81),
 ('4656', 49)]

# Forming Host graph

In [6]:
# extract all hosts involved
uniquePivot= set()
pivot = "Hostname"
for item in tqdm(readfile):
    jsonobj = json.loads(item)
    if pivot in jsonobj.keys():
           uniquePivot.add(jsonobj[pivot])

uniquePivot = list(uniquePivot)
uniquePivot

100%|███████████████| 6026/6026 [00:00<00:00, 34951.71it/s]


['WORKSTATION6.theshire.local',
 'WORKSTATION5.theshire.local',
 'MORDORDC.theshire.local']

In [7]:
# for each hostname, create edge triples
pivotDict = set()
for objId, item in enumerate(readfile):
    jsonobj = json.loads(item)
    
    keysfiltered = [x for x in jsonobj.keys() if "time" not in x.lower() and "version" not in x.lower()] #filter out redundent atr
    
    for atr in keysfiltered:
        if pivot in jsonobj.keys():
            
            preprocVal = str(jsonobj[atr]) #convert value to strings as contains lists
            preprocVal = preprocVal.split('\\')[-1] # splits long string for tools and apps
            preprocVal = (preprocVal[:50] + '...') if len(preprocVal) > 50 else preprocVal # tuncate long message and hashes
            
            pivotDict.add((str(jsonobj[pivot]), 
                           preprocVal, 
                           atr, objId)) # node1, node2, linktype, objId(row number of parsed log data)
pivotDict

{('WORKSTATION5.theshire.local', 'im_msvistalog', 'SourceModuleType', 1813),
 ('WORKSTATION5.theshire.local', '60122', 'port', 1784),
 ('MORDORDC.theshire.local',
  '{5770385F-C22A-43E0-BF4C-06F5698FFBD9}',
  'ProviderGuid',
  244),
 ('WORKSTATION5.theshire.local', '-9214364837600034816', 'Keywords', 2454),
 ('WORKSTATION5.theshire.local', 'INFO', 'EventType', 3471),
 ('WORKSTATION6.theshire.local', 'svchost.exe', 'Image', 5434),
 ('WORKSTATION5.theshire.local',
  'powershell.exe -noP -sta -w 1 -enc SQBGACgAJABQAFM...',
  'Message',
  1604),
 ('WORKSTATION5.theshire.local', 'INFO', 'EventType', 5849),
 ('MORDORDC.theshire.local', '-', 'LmPackageName', 5582),
 ('MORDORDC.theshire.local', '2', 'SeverityValue', 249),
 ('WORKSTATION6.theshire.local', "['mordorDataset']", 'tags', 5028),
 ('WORKSTATION5.theshire.local',
  '{A0C1853B-5C40-4B15-8766-3CF1C58F985A}',
  'ProviderGuid',
  4587),
 ('WORKSTATION5.theshire.local', 'wec.internal.cloudapp.net', 'host', 5166),
 ('WORKSTATION5.theshire.l

In [8]:
# add the edge data in graph
G = nx.DiGraph()

for n1, n2, link, objId in pivotDict:
    G.add_edge(n1, n2, title = link, id = objId)

In [9]:
# query graph by filtering specific nodes and links
graph_host = nx.Graph(
    (
        (u, v, e) 
        for u,v,e in G.edges(data=True) 
        if 
        e['title'] == 'EventID' or
        e['title'] == 'DestinationIp' or
        e['title'] == 'port'
    ))
nt = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")
nt.show_buttons()
nt.from_nx(graph_host)
nt.show("nx.html")

In [10]:
# to query information for specific nodes
query = next((e['id'] for u, v, e in G.edges(data=True) if v == 'lsass.exe'), None)
if query is not None:
    jsonobj = json.loads( readfile[query] )
    print(jsonobj)

{'tags': ['mordorDataset'], 'TargetProcessGUID': '{297bc33e-65d0-5f2d-8207-000000000400}', '@version': '1', 'EventType': 'INFO', 'Version': 3, 'ThreadID': 4364, 'EventTime': '2020-08-07 10:32:56', 'Task': 10, 'AccountType': 'User', 'SourceProcessGUID': '{297bc33e-5c2e-5f2b-0c00-000000000400}', 'CallTrace': 'C:\\windows\\SYSTEM32\\ntdll.dll+9d104|C:\\windows\\System32\\RPCRT4.dll+2d2c4|C:\\windows\\system32\\lsasrv.dll+176ae|C:\\windows\\SYSTEM32\\SspiSrv.dll+1a72|C:\\windows\\System32\\RPCRT4.dll+76953|C:\\windows\\System32\\RPCRT4.dll+da036|C:\\windows\\System32\\RPCRT4.dll+37a4c|C:\\windows\\System32\\RPCRT4.dll+548c8|C:\\windows\\System32\\RPCRT4.dll+2c921|C:\\windows\\System32\\RPCRT4.dll+2c1db|C:\\windows\\System32\\RPCRT4.dll+1a86f|C:\\windows\\System32\\RPCRT4.dll+19d1a|C:\\windows\\System32\\RPCRT4.dll+19301|C:\\windows\\System32\\RPCRT4.dll+18d6e|C:\\windows\\System32\\RPCRT4.dll+169a5|C:\\windows\\SYSTEM32\\ntdll.dll+333ed|C:\\windows\\SYSTEM32\\ntdll.dll+34142|C:\\windows\\S

# Adding new data to knowledge graph

In [10]:
TIH = [("['mordorDataset']", "USC University", "education", "45"), ("['mordorDataset']", "Cyberdefence inc", "work", "44")]

tih_pivot = set()

for data in TIH:
    tih_pivot.add((data[0], data[1], data[2], data[3])) # node1, node2, linktype, objId(row number of parsed log data)
tih_pivot

{("['mordorDataset']", 'Cyberdefence inc', 'work', '44'),
 ("['mordorDataset']", 'USC University', 'education', '45')}

In [11]:
# add the edge data in graph
G_TIH = nx.DiGraph()

for n1, n2, link, objId in tih_pivot:
    G_TIH.add_edge(n1, n2, title = link, id = objId)

nt = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")
nt.show_buttons()
nt.from_nx(G_TIH)
nt.show("nx.html")

In [12]:
# query graph by filtering specific nodes and links
graph_ip2tih = nx.Graph(
    (
        (u, v, e) 
        for u,v,e in G.edges(data=True) 
        if 
        e['title'] == 'tags' or
        e['title'] == 'DestinationIp'
    ))
nt = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")
nt.show_buttons()
nt.from_nx(graph_ip2tih)
nt.show("nx.html")

In [16]:
U_combine = nx.compose(G_TIH, graph_ip2tih)
nt = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")
nt.show_buttons()
nt.from_nx(U_combine)
nt.show("nx.html")

# Graph Theory Stats

In [19]:
degree_dict = dict(U_combine.degree(U_combine.nodes()))

sorted_degree = sorted(degree_dict.items(), key=itemgetter(1), reverse=True)

print("Top 20 nodes by degree:")
for d in sorted_degree[:10]:
    print(d)

Top 20 nodes by degree:
("['mordorDataset']", 5)
('MORDORDC.theshire.local', 3)
('Cyberdefence inc', 1)
('USC University', 1)
('WORKSTATION5.theshire.local', 1)
('0:0:0:0:0:0:0:1', 1)
('172.18.39.5', 1)
('WORKSTATION6.theshire.local', 1)


In [20]:
sorted(nx.neighbors(U_combine, "['mordorDataset']"))

['Cyberdefence inc',
 'MORDORDC.theshire.local',
 'USC University',
 'WORKSTATION6.theshire.local']

# Get security indicator

In [21]:
from OTXv2 import OTXv2

otx = OTXv2("1d8396f75872c265bc22f2ff336fce3fbab8707d4473c972f73e9e734c07c74f")
# Get all the indicators associated with a pulse
indicators = otx.get_pulse_indicators("5fae9363114666838175b7f7")

indicatorList = []
for indicator in indicators:
    indicatorList.append(indicator["indicator"])
    #print(indicator["indicator"] + " - "+ indicator["type"])
indicatorList

['www.sikhforjustice.org',
 'https://sikhforjustice.org/sfj/log/sfj.php',
 'audioplayer.co',
 'www.callrecorder-pro.com',
 'mideastleaks.com',
 'http://103.220.47.104',
 'https://103.220.47.104',
 'www.alqalamwebs.com',
 'ns1.rhc-jo.com',
 'ns2.rhc-jo.com',
 'rhc-jo.com',
 'http://195.123.226.249',
 'https://195.123.226.249',
 'middleeastleaks.com',
 'ustreamix.to',
 'http://217.147.169.162',
 'http://middleeastleaks.com/',
 'https://217.147.169.162',
 'https://ustreamix.to/stream.php?id=sfr-sport-3',
 'healthclubfun.com',
 'ns1.healthclubfun.com',
 'ns2.healthclubfun.com',
 'http://82.221.100.74',
 'https://82.221.100.74',
 'talktoulive.com',
 'https://www.talktoulive.com/talk/toyou/registration/test.php',
 'msft.me',
 'ppenev.com',
 'www.msft.me',
 'http://87.120.37.208',
 'http://87.120.37.208:2082',
 'https://87.120.37.208',
 'toysforislam.com',
 'https://www.toysforislam.com/aks/log/mtube_api.php/gif',
 'manage.myappie.co',
 'www.manage.myappie.co',
 'http://185.244.150.119',
 'ht

In [22]:
for tool in set(tools):
    for indicats in indicatorList:
        if tool in indicats:
            print("Mallicious tool detected: " + tool)
            print(indicats)

Mallicious tool detected: 5
http://195.123.226.249
Mallicious tool detected: 5
https://195.123.226.249
Mallicious tool detected: 5
http://185.244.150.119
Mallicious tool detected: 5
http://185.244.150.119:2082
Mallicious tool detected: 5
https://185.244.150.119
Mallicious tool detected: 5
http://185.122.58.133
Mallicious tool detected: 5
https://185.122.58.133
Mallicious tool detected: 5
http://195.123.212.82
Mallicious tool detected: 5
https://195.123.212.82
Mallicious tool detected: 5
http://185.228.232.118
Mallicious tool detected: 5
https://185.228.232.118
Mallicious tool detected: 5
http://87.120.254.170
Mallicious tool detected: 5
https://87.120.254.170
Mallicious tool detected: 5
http://85.254.144.31
Mallicious tool detected: 5
https://85.254.144.31
Mallicious tool detected: 5
v10548.dh.net.ua
Mallicious tool detected: 5
http://185.161.208.121
Mallicious tool detected: 5
http://185.161.208.121:2082
Mallicious tool detected: 5
https://185.161.208.121
Mallicious tool detected: 5
h

In [23]:
result = otx.get_indicator_details_full(IndicatorTypes.URL, "http://cyroonline.com/kitsit/lsass.exe")

NameError: name 'IndicatorTypes' is not defined

In [24]:
hashval = list(result['url_list'].values())
hashval = list(hashval[0][0].values())
hashval = list(hashval[3].values())
hashval = hashval[-1]
hashval = hashval['md5']

NameError: name 'result' is not defined

In [25]:
result = otx.get_indicator_details_full(IndicatorTypes.FILE_HASH_MD5, hashval)
result

NameError: name 'IndicatorTypes' is not defined

In [26]:
from attackcti import attack_client

# lift = attack_client()
# all_enterprise = lift.get_enterprise()
# all_enterprise.keys()

In [27]:
from pyattck import Attck
attack = Attck()

In [28]:
for technique in attack.enterprise.techniques:
    if "LSASS.exe" in technique.detection:
        print(technique.id)
#     print(technique.name)
#     for subtechnique in technique.subtechniques:
#         print(subtechnique.id)
#         print(subtechnique.name)

T1003
